# Instructions (when setting this up for a new quarter)
1. In Canvas, make an assignment called polleverywhere that has as many points as the percentage of final grade, which should also be equal the value of polleverywhere scores. 
2. In 'Grades' (in Canvas), select 'Actions: Export' to download a csv file.
3. Rename that file as 'canvas.csv' and move it to the root path (defined below).
4. Open canvas.csv in excel, or other such app, and copy the name of the column that holds your polleverywhere scores. Paste that name in the cell below (in quotes) for the cn_fieldname variable.
5. Place all polleverywhere CSV files in the folder addressed by csv_path in the cell below. Polleverywhere allows exporting in a hierarchical way, so you could have just one file. However, it's probably easier to keep things organized by periodically adding a new CSV file, rather than overwriting the old one with a bigger csv file.
6. In the cell below ("Inititialize packages . . "), update all paths and variables.
7. Run all cells to generate 'canvas_upload.csv'
8. In 'Grades', select 'Actions: Import' to upload the scores and follow the instructions.

# Tips (once set up)
1. Be sure to upload a new canvas.csv file periodically to keep pace with changes in the roster
2. Keep adding CSV files from polleverywhere into the 'polleverywhere CSV files' folder, as they are generated
3. Switching between users will require changing the root path, so comment out the root path and define your own when you switch 
4. Don't forget to download canvas_upload.csv after running the code (give it a minute or two to update first) and then upload it to Canvas to update the scores.
5. Note that the 'Grade' scores from polleverywhere are normalized by the 'participation' score, so it's okay if some of the questions do not end up getting used in class.

# Initialize packages, paths, and parameters

In [5]:
import pandas as pd
import os
import glob
import numpy as np

# Change this to the path for the current version of the course
if os.path.isdir('/Users/mmchenry/Documents/Teaching'):
    root = '/Users/mmchenry/Documents/Teaching/E109/2022 Spring E109'
elif os.path.isdir('/content/drive/MyDrive/Teaching'):
    root = '/content/drive/MyDrive/Teaching/E109 Human Physiology/e109 S2022'
else:
    raise ValueError('root path not found. Need to add new root path.')

# Create a folder within root that holds all the csv files from polleverywhere for the quarter
csv_path = root + os.path.sep + "polleverywhere csv files"

# Path to canvas CSV file that includes updated roster
cn_path = root + os.path.sep + "canvas.csv"

# Path to output csv file
out_path = root + os.path.sep + "canvas_upload.csv"

# Proportion of grade earned just from participation
propPart = 0.75

# Value of polleverywhere questions to final grade points
pollValue = 5

# Name of assignment from canvas csv file
cn_fieldname = "polleverywhere (956766)"


# Calculate polleverywhere scores from csv files, save to canvas csv file

In [10]:
# csv_list = os.listdir(csv_path)

# Open and read canvas data, capture listing of login IDs
cn_file = open(cn_path)
c = pd.read_csv(cn_file)

# From the canvas file, capture student IDs and the polleverywhere data
cnID = c.xs("SIS Login ID",axis=1)
# cnPE = c.xs(cn_fieldname,axis=1)

# Listing of csv files
csv_list = glob.glob(csv_path + os.path.sep + '*.csv')


# Loop thru each student in canvas roster
for i in range(2,len(c)):

    # Current student id in c
    curr_id = c["SIS Login ID"][i]

    # Containers for scores & participation & max possible score
    scores = np.array([0], dtype = float)
    part = np.array([0], dtype = float)
    # maxScore = np.array([len(csv_list)*100], dtype=np.float32)
    maxScore = np.array([0], dtype = float)

    # Loop thru each csv file from polleverywhere
    for fPath in csv_list:

        # Open and import data from csv file into d
        file = open(fPath)
        d = pd.read_csv(file)
        file.close()

        # Add max participation score from current csv file
        maxScore = np.append(maxScore, max(np.asarray(d.Participation[:-2], dtype = float)))

        # Loop thru each row in d
        for j in range(len(d)-2):
        # # Close input csv files

            # Eactract UCINetID from email address
            ucinetid = d.Email[j][:-8]

            # If current row matches 
            if ucinetid==curr_id:
                
                # Extract score and convert to float type
                currScore = np.asarray(d.Grade[j], dtype = float)
                currPart = np.asarray(d.Participation[j], dtype = float)

                # Add values
                scores = np.append(scores,currScore.astype(float))
                part = np.append(part,currPart.astype(float))

                # Clear temp variables
                del currScore, currPart

                # Stop the loop
                break

    # Grades for participation and correct answers
    partGrade = np.sum(part) / np.sum(maxScore) * propPart * pollValue
    corrGrade = np.sum(scores) / np.sum(maxScore)  * (1-propPart) * pollValue

    # Convert total score to strin
    scoreStr = np.array2string(partGrade + corrGrade, precision=2)

    # Log total score for current student, without brackets
    c.loc[i,cn_fieldname] = scoreStr[1:-1]
        
    print("Completed " + str(i-1) + " of " + str(len(c)-2) + " students") 


# Transfer only the necessary columns
cOut = c[['Student','ID','SIS User ID','SIS Login ID','Section',cn_fieldname]]

# Write to output file
cOut.to_csv(out_path,index=False)

# Close input csv files
# file.close()
cn_file.close()

/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.array([0], dtype=np.float)
/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  part = np.array([0], dtype=np.float)
/var/folde

Completed 1 of 350 students
Completed 2 of 350 students
Completed 3 of 350 students
Completed 4 of 350 students
Completed 5 of 350 students
Completed 6 of 350 students
Completed 7 of 350 students
Completed 8 of 350 students
Completed 9 of 350 students
Completed 10 of 350 students
Completed 11 of 350 students
Completed 12 of 350 students
Completed 13 of 350 students
Completed 14 of 350 students
Completed 15 of 350 students
Completed 16 of 350 students
Completed 17 of 350 students
Completed 18 of 350 students
Completed 19 of 350 students
Completed 20 of 350 students
Completed 21 of 350 students
Completed 22 of 350 students
Completed 23 of 350 students
Completed 24 of 350 students
Completed 25 of 350 students
Completed 26 of 350 students
Completed 27 of 350 students
Completed 28 of 350 students
Completed 29 of 350 students
Completed 30 of 350 students
Completed 31 of 350 students
Completed 32 of 350 students
Completed 33 of 350 students
Completed 34 of 350 students
Completed 35 of 350 stu

/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.array([0], dtype=np.float)
/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  part = np.array([0], dtype=np.float)
/var/folde

Completed 42 of 350 students
Completed 43 of 350 students
Completed 44 of 350 students
Completed 45 of 350 students
Completed 46 of 350 students
Completed 47 of 350 students
Completed 48 of 350 students
Completed 49 of 350 students
Completed 50 of 350 students
Completed 51 of 350 students
Completed 52 of 350 students
Completed 53 of 350 students
Completed 54 of 350 students
Completed 55 of 350 students
Completed 56 of 350 students
Completed 57 of 350 students
Completed 58 of 350 students
Completed 59 of 350 students
Completed 60 of 350 students
Completed 61 of 350 students
Completed 62 of 350 students
Completed 63 of 350 students
Completed 64 of 350 students
Completed 65 of 350 students
Completed 66 of 350 students
Completed 67 of 350 students
Completed 68 of 350 students
Completed 69 of 350 students
Completed 70 of 350 students
Completed 71 of 350 students
Completed 72 of 350 students
Completed 73 of 350 students
Completed 74 of 350 students
Completed 75 of 350 students
Completed 76 o

/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.array([0], dtype=np.float)
/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  part = np.array([0], dtype=np.float)
/var/folde

...

/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.array([0], dtype=np.float)
/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  part = np.array([0], dtype=np.float)
/var/folde

Completed 315 of 350 students
Completed 316 of 350 students
Completed 317 of 350 students
Completed 318 of 350 students
Completed 319 of 350 students
Completed 320 of 350 students
Completed 321 of 350 students
Completed 322 of 350 students
Completed 323 of 350 students
Completed 324 of 350 students
Completed 325 of 350 students
Completed 326 of 350 students
Completed 327 of 350 students
Completed 328 of 350 students
Completed 329 of 350 students
Completed 330 of 350 students
Completed 331 of 350 students
Completed 332 of 350 students
Completed 333 of 350 students
Completed 334 of 350 students
Completed 335 of 350 students
Completed 336 of 350 students
Completed 337 of 350 students
Completed 338 of 350 students
Completed 339 of 350 students
Completed 340 of 350 students
Completed 341 of 350 students


/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.array([0], dtype=np.float)
/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  part = np.array([0], dtype=np.float)
/var/folde

Completed 342 of 350 students
Completed 343 of 350 students
Completed 344 of 350 students
Completed 345 of 350 students
Completed 346 of 350 students
Completed 347 of 350 students
Completed 348 of 350 students
Completed 349 of 350 students
Completed 350 of 350 students


/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:22: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  scores = np.array([0], dtype=np.float)
/var/folders/90/3jb02y3516l0d2ygj4cshjxh0000gn/T/ipykernel_9183/283278207.py:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  part = np.array([0], dtype=np.float)
/var/folde